In [ ]:
#| default_exp ECMs.two_layer

In [ ]:
#| export
import numpy as np
from projective_simulation.ECMs.abstract_ECM import ECM
from projective_simulation.utils import _softmax

class Two_Layer(ECM):
    def __init__(self, 
                 num_actions: int, # The number of available actions.
                 glow: float, # The glow (or eta) parameter. 
                 damp: float, # The damping (or gamma) parameter. 
                 softmax: float # The softmax (or beta) parameter.
                ):

        """
        Simple, 2-layered ECM. We initialize an h-matrix with a single row of `num_actions` 
        entries corresponding to a dummy percept clip being connected to all possible actions with h-values of all 1. We 
        initialize a g-matrix with a single row of `num_actions` entries with all 0s corresponding to the *glow* values 
        of percept-action transitions.

        percepts must be created from new observations with a preprocessor, e.g. add_percepts
                      
        NOTE: This simple version misses some features such as clip deletion, emotion tags or generalization mechanisms.
        
        """

        self.num_actions = num_actions
        self.glow = glow
        self.damp = damp
        self.softmax = softmax
        #int: current number of percepts.
        self.num_percepts = 0
        #np.ndarray: h-matrix with current h-values. Defaults to all 1.
        self.hmatrix = np.ones([1,self.num_actions])
        #np.ndarray: g-matrix with current glow values. Defaults to all 0.
        self.gmatrix = np.zeros([1,self.num_actions])
        #dict: Dictionary of percepts as {"percept": index}
        self.percepts = {}

    def deliberate(self, percept: str):
        """
        Given a percept, returns an action and changes the ECM if necessary
        First, if the percept is new, it will be added to the ECM
        Then, an action is selected as a function of the percept and the h-values of edges connected to that percept
        Finally, the g-matrix is updated based on the realized percept-action pair.
        """
        #Add percept to ECM if not already present
        self.add_percept(percept)
        #Perform Random Walk
        # get index from dictionary entry
        percept_index = self.percepts[percept]
        # get h-values
        h_values = self.hmatrix[percept_index]
        # get probabilities from h-values through a softmax function
        prob = _softmax(self.softmax, h_values)
        # get action
        action = np.random.choice(range(self.num_actions), p=prob)        
        #pdate g-matrix
        self.gmatrix[int(percept_index),int(action)] = 1.
        return action

    def add_percept(self, percept):
        '''
        checks if percept is in dictionary and adds to ECM in not
        '''
        if percept not in self.percepts.keys(): 
            self.percepts[percept] = self.num_percepts
            # increment number of percepts
            self.num_percepts += 1
            # add column to h-matrix
            self.hmatrix = np.append(self.hmatrix, 
                                     np.ones([1,self.num_actions]),
                                     axis=0)
            # add column to g-matrix
            self.gmatrix = np.append(self.gmatrix, 
                                    np.zeros([1,self.num_actions]),
                                    axis=0)